# Data Engineering / Platform Review on Redshift Connection

This notebook is to show how Redshift can be connected to Databricks through Lakehouse Federation. 


For more info, check out the documentation on running federated queries on Amazon Redshift ([link](https://docs.databricks.com/aws/en/query-federation/redshift)).

![](images/Catalog_Redshift_Foreign.png)

![](images/Redshift_Table_in_DBX.png)

![](images/Redshift_Connection_Details.png)

![](images/Redshift_Catalog_Details.png)

![](images/External_Data_Catalog_image.png)

## Basic profiling query

In [0]:
USE CATALOG joy_redshift_catalog;
USE SCHEMA amenon;

In [0]:
SELECT *
FROM joy_redshift_catalog.amenon.nyctaxi_yellow_100k_daily_revenue_vw
ORDER BY order_date DESC
LIMIT 10;


order_date,gross_fare,num_rides
2009-11-30,29107.79999999998,2783
2009-11-29,29392.300000000003,2903
2009-11-28,29975.600000000006,3065
2009-11-27,21803.399999999998,2352
2009-11-26,21368.900000000005,2019
2009-11-25,31211.10000000001,3003
2009-11-24,32590.300000000003,3130
2009-11-23,27626.6,2841
2009-11-22,38094.199999999975,3662
2009-11-21,43854.39999999999,4432


In [0]:
%python
df = spark.table("joy_redshift_catalog.amenon.nyctaxi_yellow_100k_daily_revenue_vw")
display(df.limit(10))

order_date,gross_fare,num_rides
2009-11-01,38118.4,3542
2009-11-02,31126.100000000017,3047
2009-11-03,30363.300000000017,2974
2009-11-04,32542.45,3182
2009-11-05,37084.31000000001,3656
2009-11-06,38259.36000000001,3753
2009-11-07,44529.59999999997,4478
2009-11-08,37826.54999999997,3606
2009-11-09,27801.999999999996,2724
2009-11-10,31903.4,3124


## Time travel for external catalogs

A Redshift foreign catalog is just a virtual mirror of an external database. Databricks does not manage that storage or keep a Delta-style history for it.

Lakehouse Federation provides read-only access. That being said, Databricks is not the system of record and does not rewrite or version Redshift data.

Because Databricks never creates its own snapshots/versions for those foreign tables, there is no Delta/UC history to show, so DESCRIBE HISTORY and Delta time travel are not supported for foreign catalogs like Redshift or Glue.

## UPDATE / DELETE on Foreign Catalogs

Lakehouse Federation foreign catalogs (Redshift, etc.) are read-only from Databricks’ perspective. So updating and deleting would be prohibited. Also this is true for clustering, compaction, history retention, vacuum, and optimization.

Only SELECT (and some metadata operations like SHOW TABLES) are supported on foreign catalogs.

“Writes and ACID are handled by Redshift; Databricks, via Lakehouse Federation, always reads a fully committed snapshot, not partial changes.”



## Writing back to Redshift

Writing back to Redshift can be done with a jdbc connection. I'll include an example.

In [0]:
%python
# This is only an example. 

jdbc_url = (
    "jdbc:redshift:iam://mycluster.abc123.us-east-1.redshift.amazonaws.com:5439/dev"
    "?DbUser=etl_user&ssl=true"
)

temp_s3_dir = "s3a://my-redshift-temp-bucket/tmp/"
aws_iam_role_arn = "arn:aws:iam::123456789012:role/redshift-s3-access-role"

(
    df_curated.write
      .format("io.github.spark_redshift_community.spark.redshift")
      .option("url", jdbc_url)              # IAM URL, no static password
      .option("dbtable", "public.sales_curated")
      .option("tempdir", temp_s3_dir)
      .option("aws_iam_role", aws_iam_role_arn)  # role Redshift assumes to read/write S3
      .mode("overwrite")
      .save()
)


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-7680324554722852>, line 12
      8 temp_s3_dir = "s3a://my-redshift-temp-bucket/tmp/"
      9 aws_iam_role_arn = "arn:aws:iam::123456789012:role/redshift-s3-access-role"
     11 (
---> 12     df_curated.write
     13       .format("io.github.spark_redshift_community.spark.redshift")
     14       .option("url", jdbc_url)              # IAM URL, no static password
     15       .option("dbtable", "public.sales_curated")
     16       .option("tempdir", temp_s3_dir)
     17       .option("aws_iam_role", aws_iam_role_arn)  # role Redshift assumes to read/write S3
     18       .mode("overwrite")
     19       .save()
     20 )

NameError: name 'df_curated' is not defined